In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("data/voting_VA.csv")

print(df.head())
print(df.isnull().sum())

   Unnamed: 0  year     state state_po county_name  county_fips        office  \
0       11161  2000  VIRGINIA       VA    ACCOMACK        51001  US PRESIDENT   
1       11162  2000  VIRGINIA       VA    ACCOMACK        51001  US PRESIDENT   
2       11163  2000  VIRGINIA       VA    ACCOMACK        51001  US PRESIDENT   
3       11164  2000  VIRGINIA       VA    ACCOMACK        51001  US PRESIDENT   
4       11165  2000  VIRGINIA       VA   ALBEMARLE        51003  US PRESIDENT   

        candidate       party  candidatevotes  totalvotes   version   mode  
0         AL GORE    DEMOCRAT            5092       11925  20220315  TOTAL  
1  GEORGE W. BUSH  REPUBLICAN            6352       11925  20220315  TOTAL  
2     RALPH NADER       GREEN             220       11925  20220315  TOTAL  
3           OTHER       OTHER             261       11925  20220315  TOTAL  
4         AL GORE    DEMOCRAT           16255       36846  20220315  TOTAL  
Unnamed: 0        0
year              0
state      

In [2]:
# Remove the 'Unnamed: 0' column
df.drop('Unnamed: 0', axis=1, inplace=True)

# Check unique values for certain columns to ensure consistency
print(df['state'].unique())  # Should only contain "VIRGINIA"
print(df['state_po'].unique())  # Should only contain "VA"
print(df['office'].unique())  # Should only contain "US PRESIDENT"
print(df['mode'].unique())  # Check if any action is needed

['VIRGINIA']
['VA']
['US PRESIDENT']
['TOTAL' 'ABSENTEE' 'ELECTION DAY' 'PROVISIONAL']


In [3]:
df['mode'].head()

0    TOTAL
1    TOTAL
2    TOTAL
3    TOTAL
4    TOTAL
Name: mode, dtype: object

In [4]:
# If we're interested in overall voting results without differentiating between the modes:
# df_total = df[df['mode'] == 'TOTAL']
# df_total.to_csv("data/clean_total_voting_VA.csv", index=False) 

# Otherwise, we can keep all of the modes of voting and use them as features
# in our models
df['mode'] = df['mode'].astype('category')
df['mode'].head()

0    TOTAL
1    TOTAL
2    TOTAL
3    TOTAL
4    TOTAL
Name: mode, dtype: category
Categories (4, object): ['ABSENTEE', 'ELECTION DAY', 'PROVISIONAL', 'TOTAL']

In [5]:
# Provides a dataframe where the modes aggregates all individual records across different voting modes 
# into a single record per candidate per county per election year. 

df_aggregated = df.groupby(['year', 'county_name', 'county_fips', 'candidate', 'party']).agg({
    'candidatevotes': 'sum',
    'totalvotes': 'max'  # Assuming totalvotes is the same across all modes, otherwise sum might be needed
}).reset_index()

df_aggregated.head()

,year,county_name,county_fips,candidate,party,candidatevotes,totalvotes
0,2000,ACCOMACK,51001,AL GORE,DEMOCRAT,5092,11925
1,2000,ACCOMACK,51001,GEORGE W. BUSH,REPUBLICAN,6352,11925
2,2000,ACCOMACK,51001,OTHER,OTHER,261,11925
3,2000,ACCOMACK,51001,RALPH NADER,GREEN,220,11925
4,2000,ALBEMARLE,51003,AL GORE,DEMOCRAT,16255,36846


In [6]:
# county_adjacencies.csv cleaning

df2 = pd.read_csv("data/county_adjacencies.csv")

print(df2.head())
print(df2.isnull().sum())

           County  Population2022   FIPS  District                   N1  \
0        Accomack           33191  51001         2       Northumberland   
1       Albemarle          114534  51003         5  CharlottesvilleCity   
2  AlexandriaCity          155525  51510         8              Fairfax   
3       Alleghany           14835  51005         6                Craig   
4          Amelia           13455  51007         5         PrinceEdward   

           N2         N3             N4           N5        N6      N7  \
0   Lancaster  Middlesex        Mathews  Northampton       NaN     NaN   
1      Nelson    Augusta     Rockingham       Greene    Orange  Louisa   
2   Arlington        NaN            NaN          NaN       NaN     NaN   
3        Bath  Botetourt  CovingtonCity   Rockbridge       NaN     NaN   
4  Cumberland   Powhatan   Chesterfield    Dinwiddie  Nottoway     NaN   

         N8          N9     N10  N11  N12  
0       NaN         NaN     NaN  NaN  NaN  
1  Fluvanna  Buc

In [7]:
# Lots of NaNs for the adjacent neighborhoods

# Fill missing neighbor columns with "None"
neighbor_columns = ['N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'N10', 'N11', 'N12']
df2[neighbor_columns] = df2[neighbor_columns].fillna("None")

print(df2.isnull().sum())

County            0
Population2022    0
FIPS              0
District          0
N1                0
N2                0
N3                0
N4                0
N5                0
N6                0
N7                0
N8                0
N9                0
N10               0
N11               0
N12               0
dtype: int64


In [8]:
# save this dataset as a csv
# df2.to_csv("data/clean_county_adjacencies.csv", index=False)

In [18]:
import pandas as pd
#This is attempt to use county data from the nominal csv file imported into the dataframe. Note that this is dependent on year and county, and is actually pretty hard to do. 
df_concatenated = pd.read_csv('data/county_data/0002_ts_nominal_county.csv')

updated_rows = []

for _, row in df_aggregated.iterrows():
    county_name = row['county_name']
    
    county_row = df_concatenated.loc[df_concatenated['COUNTY'].contains(county_name)]
    
    if not county_row.empty:
        recent_year_columns = [col for col in year_columns if str(most_recent_year) in col]
        
        county_data = county_row[recent_year_columns].iloc[0]
        
        updated_row = pd.concat([row, county_data])
        updated_rows.append(updated_row)
    else:
        print("Not found")
        updated_rows.append(row)

df_updated = pd.DataFrame(updated_rows)

print(df_updated)

/tmp/ipykernel_283018/3592115686.py:4: DtypeWarning: Columns (3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,

ValueError: invalid literal for int() with base 10: 'A125'

In [11]:
#This is another attempt at getting info from csv files into the dataframe. 
csv_files = ["project_voting/data/county_data/0002_ds176_20105_county_E.csv", "project_voting/data/county_data/0002_ds191_20125_county_E.csv", "project_voting/data/county_data/0002_ds191_20125_county_E.csv", "project_voting/data/county_data/0002_ds191_20125_county_E.csv", "project_voting/data/county_data/0002_ds239_20185_county_E.csv", "project_voting/data/county_data/0002_ds249_20205_county_E.csv "]

updated_rows = []

for _, row in df_aggregated.iterrows():
    year = row['year']
    county_name = row['county_name']
    if(year<
    csv_file = next((file for file in csv_files if int(file.split('-')[0]) <= year <= int(file.split('-')[1])), None)
    
    if csv_file:
        df_temp = pd.read_csv(csv_file)
        
        county_row = df_temp.loc[df_temp['county_name'] == county_name]
        
        if not county_row.empty:
            updated_row = pd.concat([row, county_row.iloc[0]])
            updated_rows.append(updated_row)
        else:
            updated_rows.append(row)
    else:
        updated_rows.append(row)

df_updated = pd.DataFrame(updated_rows)

print(df_updated)

year
county_name
county_fips
candidate
party
candidatevotes
totalvotes
